In [3]:
import tensorflow as tf
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


In [2]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.6 MB/s eta 0:00:00


# Loading data

In [4]:
data_fake_reviews = pd.read_csv('./fake reviews dataset.csv')

In [5]:
data_fake_reviews.shape

(40432, 4)

In [6]:
data_fake_reviews.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [10]:
data_fake_reviews['category'].unique()

array(['Home_and_Kitchen_5', 'Sports_and_Outdoors_5', 'Electronics_5',
       'Movies_and_TV_5', 'Tools_and_Home_Improvement_5',
       'Pet_Supplies_5', 'Kindle_Store_5', 'Books_5', 'Toys_and_Games_5',
       'Clothing_Shoes_and_Jewelry_5'], dtype=object)

In [8]:
data_fake_reviews.label.unique()

array(['CG', 'OR'], dtype=object)

In [11]:
data_fake_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40432 entries, 0 to 40431
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   category  40432 non-null  object 
 1   rating    40432 non-null  float64
 2   label     40432 non-null  object 
 3   text_     40432 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.2+ MB


In [9]:
data_fake_reviews.columns

Index(['category', 'rating', 'label', 'text_'], dtype='object')

In [12]:
df_OR = data_fake_reviews[data_fake_reviews['label'] == 'OR']


In [13]:
df_CG = data_fake_reviews[data_fake_reviews['label'] == 'CG']

In [14]:
df_OR_sample = df_OR.sample(n=2000)

In [15]:
fake_reviews_true = df_OR_sample[['text_','label']]

In [16]:
fake_reviews_true

,text_,label
22419,I have two outside cats and this works great f...,OR
14592,When I first saw this movie I was blown away b...,OR
2898,"The ""polish"" already shows lots of scratches, ...",OR
1167,"A nice idea that my daughter loved, too bad th...",OR
30440,The book covers many ways to build and maintai...,OR
...,...,...
33322,This was okay. Kids played with it a bit but t...,OR
11857,"I'm very disappointed in this drive, as even w...",OR
6156,"this bota bag is awsome , perfect size , stron...",OR
16147,Easy to install the shower head is great to cl...,OR


In [17]:
fake_reviews_gpt = pd.read_csv('./fake_reviews_gpt.csv')

In [18]:
fake_reviews_gpt.head()

,text_,label
0,The 'WinterWonder' woolen hat is warm and cozy...,CG
1,The 'SmoothMix' blender makes delicious smooth...,CG
2,I'm very satisfied with this distance learning...,CG
3,The movie 'Nebula Nebulae' is a visually stunn...,CG
4,I'm very satisfied with this electric shaver. ...,CG


In [19]:
# Combine the dataframes
combined_df = pd.concat([fake_reviews_true, fake_reviews_gpt])

# Shuffle the combined dataframe
shuffled_df = combined_df.sample(frac=1).reset_index(drop=True)


# Obtain the training dataset and validation dataset

In [20]:
dict_target = {'CG':0, 'OR':1}
def encode_label(x):
    return dict_target.get(x,-1)

In [21]:
data_fake_reviews["target"] = data_fake_reviews["label"].apply(lambda x: encode_label(x))

In [22]:
shuffled_df["target"] = shuffled_df["label"].apply(lambda x: encode_label(x))

In [23]:
# Split the data into training and validation sets
train_dataset, valid_dataset  = train_test_split(shuffled_df, test_size=0.2, shuffle=True, stratify=None, random_state=2021)

In [24]:
train_dataset.shape

(3200, 3)

In [25]:
# Split the data into training and validation sets
train_dataset_new, tv_dataset = train_test_split(data_fake_reviews, test_size=0.4, shuffle=True, stratify=None, random_state=2021)

In [26]:
test_dataset_raw, valid_dataset_new = train_test_split(tv_dataset, test_size=0.5, shuffle=True, stratify=None, random_state=2021)

In [27]:
train_dataset_new.shape

(24259, 5)

In [28]:
train_dataset_new_true = train_dataset_new[['text_','label','target']]

In [29]:
fake_reviews_gpt["target"] = fake_reviews_gpt["label"].apply(lambda x: encode_label(x))

In [30]:
fake_reviews_gpt

,text_,label,target
0,The 'WinterWonder' woolen hat is warm and cozy...,CG,0
1,The 'SmoothMix' blender makes delicious smooth...,CG,0
2,I'm very satisfied with this distance learning...,CG,0
3,The movie 'Nebula Nebulae' is a visually stunn...,CG,0
4,I'm very satisfied with this electric shaver. ...,CG,0
...,...,...,...
1995,"'Planetary Plushies' are soft, colorful, and e...",CG,0
1996,The TV series 'Galactic Guardians' is a superh...,CG,0
1997,This virtual reality gaming service is impress...,CG,0
1998,The movie 'Astral Alliances' is an action-pack...,CG,0


In [31]:
# Combine the dataframes
combined_df = pd.concat([train_dataset_new_true, fake_reviews_gpt])

# Shuffle the combined dataframe
train_dataset = combined_df.sample(frac=1).reset_index(drop=True)

In [32]:
train_dataset.shape

(26259, 3)

In [33]:
train_dataset_0 = train_dataset_new[train_dataset_new['target']==0]

In [34]:
train_dataset_0.shape

(12178, 5)

In [35]:
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [36]:
# Tokenize the data
train_encodings = tokenizer(train_dataset['text_'].tolist(), truncation=True, padding=True, max_length=256)
valid_encodings = tokenizer(valid_dataset_new['text_'].tolist(), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_dataset_raw['text_'].tolist(), truncation=True, padding=True, max_length=256)

In [37]:
# Convert the data to TensorFlow tensors
train_dataset_tf = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_dataset['target'].values
))
valid_dataset_tf = tf.data.Dataset.from_tensor_slices((
    dict(valid_encodings),
    valid_dataset_new['target'].values
))


In [38]:
test_dataset_tf = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_dataset_raw['target'].values
))

# Load the model

In [39]:
# Load the model
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [40]:

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:

# Train the model
model.fit(train_dataset_tf.shuffle(1000).batch(16), epochs=1, batch_size=16, validation_data=valid_dataset_tf.batch(16))

1642/1642 [==============================] - 1591s 920ms/step - loss: 0.1425 - accuracy: 0.9361 - val_loss: 0.0915 - val_accuracy: 0.9620


# evaluate the performance

In [44]:
from sklearn.metrics import classification_report
import numpy as np
# Make predictions on the validation dataset
test_dataset_batched = test_dataset_tf.batch(16)
y_pred_raw = model.predict(test_dataset_batched)

# Convert raw predictions to class predictions
y_pred = np.argmax(y_pred_raw.logits, axis=1)

# Get the actual class labels
y_true = []
for features, label in test_dataset_tf:
    y_true.append(label.numpy())
y_true = np.array(y_true)


# Generate a classification report
report = classification_report(y_true, y_pred, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report)


506/506 [==============================] - 140s 269ms/step
              precision    recall  f1-score   support

          CG     0.9319    0.9947    0.9623      3974
          OR     0.9945    0.9297    0.9610      4112

    accuracy                         0.9617      8086
   macro avg     0.9632    0.9622    0.9617      8086
weighted avg     0.9637    0.9617    0.9616      8086



In [ ]:
model.save('my_model_fake_reviews')



# Test dataset with 150


In [45]:
test_dataset_raw.shape

(8086, 5)

In [46]:
test_dataset_raw.head()

,category,rating,label,text_,target
31179,Books_5,5.0,CG,Rick Bragg is one of the most entertaining cha...,0
26353,Kindle_Store_5,5.0,CG,Ms. Rasley's prose never fails to capture the ...,0
9403,Electronics_5,5.0,OR,Rebuilt my PC from the ground up and I absolut...,1
13753,Movies_and_TV_5,5.0,CG,An awesome show! Got a few of the movies on dv...,0
19789,Pet_Supplies_5,5.0,OR,"Highly recommend this brand of milk replacer, ...",1


In [47]:
test_dataset_raw_0 = test_dataset_raw[test_dataset_raw['target']==0]

In [48]:
test_dataset_raw_0.shape

(3974, 5)

In [94]:
random_50_rows = test_dataset_raw_0.sample(n=50)

# Build the new DataFrame using the randomly selected rows
test_dataset_raw_50 = pd.DataFrame(random_50_rows)

In [95]:
test_dataset_raw_50.head()

,category,rating,label,text_,target
30520,Books_5,4.0,CG,"This could not be called a book, but it is a b...",0
1919,Home_and_Kitchen_5,5.0,CG,Used for moving kitchen ware. Works well. Ea...,0
24604,Kindle_Store_5,5.0,CG,Loved the awkwardness of the time period and t...,0
24184,Kindle_Store_5,5.0,CG,"Never a dull moment. Not a bad story, just not...",0
39193,Clothing_Shoes_and_Jewelry_5,5.0,CG,I got these to wear with a T-shirt. I normally...,0


In [96]:
test_dataset_raw_1 = test_dataset_raw[test_dataset_raw['target']==1]

In [97]:
random_1_rows = test_dataset_raw_1.sample(n=100)

# Build the new DataFrame using the randomly selected rows
test_dataset_raw_1_100 = pd.DataFrame(random_1_rows)

In [98]:
test_dataset_raw_1_100.shape

(100, 5)

In [99]:
merged_df = pd.concat([test_dataset_raw_50, test_dataset_raw_1_100], ignore_index=True)

# Shuffle the index
test_150_df = merged_df.sample(frac=1).reset_index(drop=True)

In [100]:
test_150_df

,category,rating,label,text_,target
0,Movies_and_TV_5,5.0,OR,"Great story. B&W, full screen, monophonic soun...",1
1,Home_and_Kitchen_5,5.0,OR,Arrived as stated when ordered. Works as expe...,1
2,Books_5,4.0,CG,"Justice Louis Brandeis, appointed to the US Su...",0
3,Sports_and_Outdoors_5,5.0,OR,This is a good light hammock.\n\nIt is contain...,1
4,Pet_Supplies_5,4.0,OR,I have been using this condicioner without any...,1
...,...,...,...,...,...
145,Movies_and_TV_5,5.0,OR,I was very pleased with the quality of the vid...,1
146,Tools_and_Home_Improvement_5,5.0,OR,Works just fine for me. I don't need to move i...,1
147,Home_and_Kitchen_5,4.0,CG,Knives look well in the kitchen. The only prob...,0
148,Kindle_Store_5,5.0,CG,Loved the awkwardness of the time period and t...,0


In [101]:
test_encodings_150 = tokenizer(test_150_df['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_150 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150),
    test_150_df['target'].values
))



In [102]:
test_dataset_150_batched = test_dataset_150.batch(16)
y_pred_raw_150 = model.predict(test_dataset_150_batched)

# Convert raw predictions to class predictions
y_pred_150 = np.argmax(y_pred_raw_150.logits, axis=1)

# Get the actual class labels
y_true_150 = []
for features, label in test_dataset_150:
    y_true_150.append(label.numpy())
y_true_150 = np.array(y_true_150)


# Generate a classification report
report_150 = classification_report(y_true_150, y_pred_150, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report_150)

10/10 [==============================] - 3s 246ms/step
              precision    recall  f1-score   support

          CG     0.9259    1.0000    0.9615        50
          OR     1.0000    0.9600    0.9796       100

    accuracy                         0.9733       150
   macro avg     0.9630    0.9800    0.9706       150
weighted avg     0.9753    0.9733    0.9736       150



# Test dataset with 125

In [128]:
random_25_rows_125 = test_dataset_raw_0.sample(n=25)

# Build the new DataFrame using the randomly selected rows
test_dataset_raw_251 = pd.DataFrame(random_25_rows_125)

In [129]:
random_1_rows = test_dataset_raw_1.sample(n=100)

# Build the new DataFrame using the randomly selected rows
test_dataset_raw_1_100 = pd.DataFrame(random_1_rows)

In [130]:
merged_df_1 = pd.concat([test_dataset_raw_251, test_dataset_raw_1_100], ignore_index=True)

# Shuffle the index
test_1251_df = merged_df_1.sample(frac=1).reset_index(drop=True)

In [131]:
test_encodings_1251 = tokenizer(test_1251_df['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_1251 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_1251),
    test_1251_df['target'].values
))



In [132]:
test_dataset_1251_batched = test_dataset_1251.batch(16)
y_pred_raw_1251 = model.predict(test_dataset_1251_batched)

# Convert raw predictions to class predictions
y_pred_1251 = np.argmax(y_pred_raw_1251.logits, axis=1)

# Get the actual class labels
y_true_1251 = []
for features, label in test_dataset_1251:
    y_true_1251.append(label.numpy())
y_true_1251 = np.array(y_true_1251)


# Generate a classification report
report_1251 = classification_report(y_true_1251, y_pred_1251, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report_1251)

8/8 [==============================] - 2s 258ms/step
              precision    recall  f1-score   support

          CG     0.8276    0.9600    0.8889        25
          OR     0.9896    0.9500    0.9694       100

    accuracy                         0.9520       125
   macro avg     0.9086    0.9550    0.9291       125
weighted avg     0.9572    0.9520    0.9533       125



# chatgpt 3.5

In [133]:
index_1 = [19080, 37380,  6466,  3934, 33275, 24286, 11892, 36412,  5625,
            25570,  8501, 20411, 28867, 12409, 33851, 34246, 39476, 13831,
            24371,  9662, 33463,  2954, 36835, 11507, 17802, 28244, 10309,
            31786, 26381,  3940, 12791, 21130, 26333, 34131, 28036, 21829,
            36002, 32896, 13185,  4173,  6265, 36584, 18300, 36500,  8968,
            32790, 26048, 10131, 28084,  6407]

In [134]:
test_dataset_raw_0_50 = test_dataset_raw_0.loc[index_1]

In [135]:
test_dataset_raw_0_50.index

Int64Index([19080, 37380,  6466,  3934, 33275, 24286, 11892, 36412,  5625,
            25570,  8501, 20411, 28867, 12409, 33851, 34246, 39476, 13831,
            24371,  9662, 33463,  2954, 36835, 11507, 17802, 28244, 10309,
            31786, 26381,  3940, 12791, 21130, 26333, 34131, 28036, 21829,
            36002, 32896, 13185,  4173,  6265, 36584, 18300, 36500,  8968,
            32790, 26048, 10131, 28084,  6407],
           dtype='int64')

In [136]:
test_dataset_raw_0_50['text_'][6407]

"I now want these in a larger size. I've also used them for a few hours in a cave. The seams are solid and they have a nice feel to them. I have had no issues with the seams becoming loose."

In [137]:
test_dataset_raw_0_50['text_'][19080] = "I really admire the Skeletool; it's the first saw I've ever owned, and while it may be a bit pricey, it's definitely worth the investment. The clean cuts it delivers are impressive, and I find myself using it on numerous projects. Recently, I decided to purchase a new one and used it to cut with a saw I've had for about a year now. The Skeletool's build quality is excellent, but it is quite heavy, which can be a bit challenging on my workbench. However, I've still grown fond of it. One minor issue I've encountered is that it doesn't work as effectively when pushing the handle back against the base. Nonetheless, I'm generally satisfied with its performance and reliability."

<ipython-input-137-b6051550059d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][19080] = "I really admire the Skeletool; it's the first saw I've ever owned, and while it may be a bit pricey, it's definitely worth the investment. The clean cuts it delivers are impressive, and I find myself using it on numerous projects. Recently, I decided to purchase a new one and used it to cut with a saw I've had for about a year now. The Skeletool's build quality is excellent, but it is quite heavy, which can be a bit challenging on my workbench. However, I've still grown fond of it. One minor issue I've encountered is that it doesn't work as effectively when pushing the handle back against the base. Nonetheless, I'm generally satisfied with its performance and reliability."


In [138]:
test_dataset_raw_0_50['text_'][37380] ="The belt is crafted with excellent craftsmanship and made from high-quality materials. The only drawback I've noticed is that it comes with a relatively small buckle."

<ipython-input-138-efaa4ca88644>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][37380] ="The belt is crafted with excellent craftsmanship and made from high-quality materials. The only drawback I've noticed is that it comes with a relatively small buckle."


In [139]:
test_dataset_raw_0_50['text_'][6466] ="This item is an ideal fit for 18-inch dolls. However, there's a minor issue with a small screw that comes with it, as it tends to be overly tight. Additionally, the clamp that holds the doll may be a bit loose, although this shouldn't pose a problem for the majority of dolls."

<ipython-input-139-de8f3bb6d6e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][6466] ="This item is an ideal fit for 18-inch dolls. However, there's a minor issue with a small screw that comes with it, as it tends to be overly tight. Additionally, the clamp that holds the doll may be a bit loose, although this shouldn't pose a problem for the majority of dolls."


In [140]:
test_dataset_raw_0_50['text_'][3934] = "I purchased the Martha Stewart pom-pom flower pot, and it fits perfectly in my kitchen, adding a lovely touch to the decor. The top features a durable stainless steel ring with a sturdy ceramic edge, making it a visually appealing piece. Although it might not be the most budget-friendly option, I find it to be a worthwhile addition to my kitchen. I received a free sample of this product at a discounted price in exchange for providing my honest and unbiased review, and I couldn't be happier with the purchase. Having used this product, I can confidently say that it has become an essential item in my kitchen. I'm delighted to have received it at a discount in exchange for my honest and unbiased review, and I genuinely believe it has been a valuable addition to my home"

<ipython-input-140-b30b2886de5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][3934] = "I purchased the Martha Stewart pom-pom flower pot, and it fits perfectly in my kitchen, adding a lovely touch to the decor. The top features a durable stainless steel ring with a sturdy ceramic edge, making it a visually appealing piece. Although it might not be the most budget-friendly option, I find it to be a worthwhile addition to my kitchen. I received a free sample of this product at a discounted price in exchange for providing my honest and unbiased review, and I couldn't be happier with the purchase. Having used this product, I can confidently say that it has become an essential item in my kitchen. I'm delighted to have received it at a discount in exchange for my honest and

In [141]:
test_dataset_raw_0_50['text_'][33275] = "I cannot express how much I appreciate this product, and it brings me immense joy to see how much my daughter loves it."

<ipython-input-141-19b2cb63559a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][33275] = "I cannot express how much I appreciate this product, and it brings me immense joy to see how much my daughter loves it."


In [142]:
test_dataset_raw_0_50['text_'][24286] ="It's just perfect for that special time of year when her sister returns for a new year, and we get a glimpse of their joy and excitement."

<ipython-input-142-7f3593c100ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][24286] ="It's just perfect for that special time of year when her sister returns for a new year, and we get a glimpse of their joy and excitement."


In [143]:
test_dataset_raw_0_50['text_'][11892] ="When using the device with my computer running Windows XP and Windows 7, I found that it connected easily without direct computer connection, requiring only a USB cable. However, when attempting to connect it to my Windows 8.1 computer with an old hard drive, I encountered some confusion and had to take a few extra steps to establish a stable connection. Overall, the device worked well with Windows XP and Windows 7, but required additional attention for proper connectivity with the Windows 8.1 computer and old hard drive setup."

<ipython-input-143-913db2992c1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][11892] ="When using the device with my computer running Windows XP and Windows 7, I found that it connected easily without direct computer connection, requiring only a USB cable. However, when attempting to connect it to my Windows 8.1 computer with an old hard drive, I encountered some confusion and had to take a few extra steps to establish a stable connection. Overall, the device worked well with Windows XP and Windows 7, but required additional attention for proper connectivity with the Windows 8.1 computer and old hard drive setup."


In [144]:
test_dataset_raw_0_50['text_'][36412] ="My four-year-old son (age 3) absolutely adores these toys, and we also have a smaller version for the baby. He loves the sounds the animals make, and it's a fantastic toy that keeps him entertained. I highly recommend it. It was a gift for my niece, and she loves it too! Getting it as a free item made it even better. The set is the perfect size and looks wonderful. Assembling it is easy, and my son enjoys playing with it. The inclusion of two small pieces, each with a lid, is a nice feature. We gave this as a Christmas gift to my son, and he adores it; it's simply adorable. This train set is fantastic, and our kids enjoy playing with it. We got it for our 6-year-old, and it's a hit. Even our 5-year-old loves it. The train is about 18 tall and weighs approximately 8 lbs, making it suitable for younger children. Furthermore, it's a sturdy and durable toy, perfect for young babies, and the small plastic cup holder is a convenient addition."

<ipython-input-144-436abdf226fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][36412] ="My four-year-old son (age 3) absolutely adores these toys, and we also have a smaller version for the baby. He loves the sounds the animals make, and it's a fantastic toy that keeps him entertained. I highly recommend it. It was a gift for my niece, and she loves it too! Getting it as a free item made it even better. The set is the perfect size and looks wonderful. Assembling it is easy, and my son enjoys playing with it. The inclusion of two small pieces, each with a lid, is a nice feature. We gave this as a Christmas gift to my son, and he adores it; it's simply adorable. This train set is fantastic, and our kids enjoy playing with it. We got it for our 6-year-old, and it's a hit.

In [145]:
test_dataset_raw_0_50['text_'][5625]  = "The perfect powder measure lives up to its name as it functions flawlessly. However, the only issue I encountered was with a small screw that was a bit too tight. Despite that, I'm still using it with great satisfaction."

<ipython-input-145-73aeffaf752e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][5625]  = "The perfect powder measure lives up to its name as it functions flawlessly. However, the only issue I encountered was with a small screw that was a bit too tight. Despite that, I'm still using it with great satisfaction."


In [146]:
test_dataset_raw_0_50['text_'][25570] = "I found great pleasure in reading this book as it beautifully combined chemistry and humor in a love story. However, I couldn't help but feel a bit melancholic when the story concluded. That might be the reason why I rated it 4 stars. The author's storytelling had me captivated throughout the entire journey."

<ipython-input-146-1ec7b74e365d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][25570] = "I found great pleasure in reading this book as it beautifully combined chemistry and humor in a love story. However, I couldn't help but feel a bit melancholic when the story concluded. That might be the reason why I rated it 4 stars. The author's storytelling had me captivated throughout the entire journey."


In [147]:
test_dataset_raw_0_50['text_'][8501] = "Two words: heavy-duty! I use it daily with my phone, and it functions exceptionally well."

<ipython-input-147-3e715dd8f0a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][8501] = "Two words: heavy-duty! I use it daily with my phone, and it functions exceptionally well."


In [148]:
test_dataset_raw_0_50['text_'][20411] ="It works wonderfully and has a pleasant scent. In fact, it's the only product I will purchase for my cats!"

<ipython-input-148-6dbe9ddfa188>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][20411] ="It works wonderfully and has a pleasant scent. In fact, it's the only product I will purchase for my cats!"


In [149]:
test_dataset_raw_0_50['text_'][28867] = "The book is highly recommended for anyone looking for a clear and easily understandable format, along with straightforward examples that are easy to follow."

<ipython-input-149-7425deab10b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][28867] = "The book is highly recommended for anyone looking for a clear and easily understandable format, along with straightforward examples that are easy to follow."


In [150]:
test_dataset_raw_0_50['text_'][12409] ="This does remind me of 'Colusses: The World's Towards the Div', but I'm not sure what you want me to rephrase. Could you please provide more context or clarify your request?"

<ipython-input-150-06b1847c36cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][12409] ="This does remind me of 'Colusses: The World's Towards the Div', but I'm not sure what you want me to rephrase. Could you please provide more context or clarify your request?"


In [151]:
test_dataset_raw_0_50['text_'][33851] ="This doll is absolutely adorable! She is incredibly soft and cuddly, and the baby's touch is gentle and tender."

<ipython-input-151-0acde49c2d75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][33851] ="This doll is absolutely adorable! She is incredibly soft and cuddly, and the baby's touch is gentle and tender."


In [152]:
test_dataset_raw_0_50['text_'][34246] ="I wish this item could have been slightly more expensive. While I'm unsure if I would have bought it from Amazon, I am contemplating my decision."

<ipython-input-152-92f7350a2ecf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][34246] ="I wish this item could have been slightly more expensive. While I'm unsure if I would have bought it from Amazon, I am contemplating my decision."


In [153]:
test_dataset_raw_0_50['text_'][39476] = "These shoes are incredibly fashionable and stylish, making anyone wearing them look well-dressed. I purchased them for my son, and he absolutely loves them! He wears them frequently, and they provide great comfort. However, it's important to note that these are not waterproof boots, so their longevity might be limited in certain conditions. Overall, they are a fantastic shoe choice."

<ipython-input-153-9047959fb319>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][39476] = "These shoes are incredibly fashionable and stylish, making anyone wearing them look well-dressed. I purchased them for my son, and he absolutely loves them! He wears them frequently, and they provide great comfort. However, it's important to note that these are not waterproof boots, so their longevity might be limited in certain conditions. Overall, they are a fantastic shoe choice."


In [154]:
test_dataset_raw_0_50['text_'][13831] = "This film is truly remarkable as a Christian movie. The storytelling is well-crafted, and the acting is of excellent quality. I highly recommend this film to anyone who appreciates a compelling Christian story. As a movie enthusiast, I must say this one stands out as a great choice."

<ipython-input-154-2caa9230c099>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][13831] = "This film is truly remarkable as a Christian movie. The storytelling is well-crafted, and the acting is of excellent quality. I highly recommend this film to anyone who appreciates a compelling Christian story. As a movie enthusiast, I must say this one stands out as a great choice."


In [155]:
test_dataset_raw_0_50['text_'][24371] = "I derive immense pleasure from reading this series. The characters are skillfully developed, and the narrative is masterfully told. The story is truly captivating and engrossing"

<ipython-input-155-436a9663ffe8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][24371] = "I derive immense pleasure from reading this series. The characters are skillfully developed, and the narrative is masterfully told. The story is truly captivating and engrossing"


In [156]:
test_dataset_raw_0_50['text_'][9662] ="This is another outstanding addition to an already impressive collection, offering excellent value for the money spent. While I may not be an expert on music, as a passionate music lover, I can confidently say that this is a remarkable choice."

<ipython-input-156-322e19cebd58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][9662] ="This is another outstanding addition to an already impressive collection, offering excellent value for the money spent. While I may not be an expert on music, as a passionate music lover, I can confidently say that this is a remarkable choice."


In [157]:
test_dataset_raw_0_50['text_'][33463] = "Our 3-year-old daughter thoroughly enjoys playing with this toy, especially in her playroom. It's a delightful addition to her playtime."

<ipython-input-157-da9e8f503cdc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][33463] = "Our 3-year-old daughter thoroughly enjoys playing with this toy, especially in her playroom. It's a delightful addition to her playtime."


In [158]:
test_dataset_raw_0_50['text_'][2954] = "This product is generously sized, providing a comfortable and pleasant feel in my hand. I purchased it as a gift for a friend who turned out to be a big fan of it. She loved it so much that she even sent one to her brother as a Christmas present. The positive reviews mentioning its excellent quality were indeed accurate, and I can vouch for its impressive build."

<ipython-input-158-11b359630d46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][2954] = "This product is generously sized, providing a comfortable and pleasant feel in my hand. I purchased it as a gift for a friend who turned out to be a big fan of it. She loved it so much that she even sent one to her brother as a Christmas present. The positive reviews mentioning its excellent quality were indeed accurate, and I can vouch for its impressive build."


In [159]:
test_dataset_raw_0_50['text_'][36835] = "Highly comfortable and pleasantly designed, I wholeheartedly recommend this product. Its quality is superb, and it offers a comfortable fit, even for those with wide feet."

<ipython-input-159-981a18f10eeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][36835] = "Highly comfortable and pleasantly designed, I wholeheartedly recommend this product. Its quality is superb, and it offers a comfortable fit, even for those with wide feet."


In [160]:
test_dataset_raw_0_50['text_'][11507] = "After enduring several months of delays, this product has finally become a reality. I rate it 4 stars only because it's slightly too small for a laptop. The keyboard size is a bit compact, which might not be a major issue for me, but it could be better suited for laptops. Additionally, the keyboard is a bit large and cumbersome to handle. Personally, I prefer having my laptop on my lap or on the couch rather than dealing with a bulky keyboard on my lap with a hard surface. There are some doubts about the keyboard's longevity, as it may be too small for a laptop. However, it does a commendable job of securely holding my Macbook Pro, and I've been using it comfortably for some time now."

<ipython-input-160-ce03be20f652>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][11507] = "After enduring several months of delays, this product has finally become a reality. I rate it 4 stars only because it's slightly too small for a laptop. The keyboard size is a bit compact, which might not be a major issue for me, but it could be better suited for laptops. Additionally, the keyboard is a bit large and cumbersome to handle. Personally, I prefer having my laptop on my lap or on the couch rather than dealing with a bulky keyboard on my lap with a hard surface. There are some doubts about the keyboard's longevity, as it may be too small for a laptop. However, it does a commendable job of securely holding my Macbook Pro, and I've been using it comfortably for some time n

In [161]:
test_dataset_raw_0_50['text_'][17802] = "I've owned this gun for a couple of years, and I couldn't be happier with its performance. Despite its small size, it packs a punch and proves its worth without the need for spending extra money. It's particularly useful for handling small jobs and comes in handy for tasks, especially when on the go or working in a truck."

<ipython-input-161-0425f2da9a42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][17802] = "I've owned this gun for a couple of years, and I couldn't be happier with its performance. Despite its small size, it packs a punch and proves its worth without the need for spending extra money. It's particularly useful for handling small jobs and comes in handy for tasks, especially when on the go or working in a truck."


In [162]:
test_dataset_raw_0_50['text_'][28244] = "I received this book as an Amazon freebie and got an ARC in exchange for an honest review. While the first two books were enjoyable, the rest of the story didn't quite captivate me, especially the second book, which lacked a compelling storyline. The hero and heroine were a bit frustrating, and their characters lacked depth, although the hero was likable and the heroine's growth was interesting. The villain was well-portrayed, but didn't evoke much emotion. The ending was pleasant, and I had hoped for a sequel that didn't materialize. Overall, the book had some enjoyable aspects with twists and turns, making it worth recommending to those who appreciate a good story."

<ipython-input-162-9c5a84f9b9ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][28244] = "I received this book as an Amazon freebie and got an ARC in exchange for an honest review. While the first two books were enjoyable, the rest of the story didn't quite captivate me, especially the second book, which lacked a compelling storyline. The hero and heroine were a bit frustrating, and their characters lacked depth, although the hero was likable and the heroine's growth was interesting. The villain was well-portrayed, but didn't evoke much emotion. The ending was pleasant, and I had hoped for a sequel that didn't materialize. Overall, the book had some enjoyable aspects with twists and turns, making it worth recommending to those who appreciate a good story."


In [163]:
test_dataset_raw_0_50['text_'][10309] ="The device serves its purpose, but it does have a design flaw. I must admit that it's a bit bulky. If you're looking for a nice and sturdy desktop, then this is the one to get. However, if you prefer a smaller desktop, then this might not be the best choice for you. I purchased this device for..."

<ipython-input-163-c356e26ba2e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][10309] ="The device serves its purpose, but it does have a design flaw. I must admit that it's a bit bulky. If you're looking for a nice and sturdy desktop, then this is the one to get. However, if you prefer a smaller desktop, then this might not be the best choice for you. I purchased this device for..."


In [164]:
test_dataset_raw_0_50['text_'][31786] ="It's unfortunate that this book has been overlooked in history, but dwelling on the past isn't worth my time. However, I must acknowledge that the book is well-written and in good condition. On the brighter side, I absolutely adored this book and couldn't wait to read it, along with its thrilling sequel. It seamlessly continues the series, and I'm eagerly looking forward to what unfolds next. This entire series has been a delightful journey for me over the past month, and each book has been a joy to read."

<ipython-input-164-07c98fa1d941>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][31786] ="It's unfortunate that this book has been overlooked in history, but dwelling on the past isn't worth my time. However, I must acknowledge that the book is well-written and in good condition. On the brighter side, I absolutely adored this book and couldn't wait to read it, along with its thrilling sequel. It seamlessly continues the series, and I'm eagerly looking forward to what unfolds next. This entire series has been a delightful journey for me over the past month, and each book has been a joy to read."


In [165]:
test_dataset_raw_0_50['text_'][26381] = "I absolutely adored this book! The emotional love story kept me captivated, and I loved the characters' journey to find their way back together. It's a must-read for all romance enthusiasts. The story was very engaging, and the characters were well-developed, which I truly appreciated."

<ipython-input-165-be9211b3276e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][26381] = "I absolutely adored this book! The emotional love story kept me captivated, and I loved the characters' journey to find their way back together. It's a must-read for all romance enthusiasts. The story was very engaging, and the characters were well-developed, which I truly appreciated."


In [166]:
test_dataset_raw_0_50['text_'][3940] ="I have two Keurig-type vacuum cleaners, and they work perfectly fine. The extra suction on the top is a great feature, and the blades make it easy to clean. I would definitely repurchase this product./The reviews mentioned that it was a great size, but I found it to be smaller than expected, so I had to order another set for my grandson. Nevertheless, I love the color, and they work wonderfully for me. /This blanket is fantastic! It's soft, cozy, and durable. While it's not overly heavy, it does a good job of keeping me comfortable in hot weather. I'll be buying another set for my daughter, who is having trouble sleeping. /I bought these curtains, and they looked pretty cute. They are also great for my son's bed. The colors are exactly as pictured, and the fabric is softer than expected, but I still like it. The vibrant colors are a pleasant surprise. They are a wonderful addition to my bedroom, and I highly recommend them."

<ipython-input-166-22d559de4195>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][3940] ="I have two Keurig-type vacuum cleaners, and they work perfectly fine. The extra suction on the top is a great feature, and the blades make it easy to clean. I would definitely repurchase this product./The reviews mentioned that it was a great size, but I found it to be smaller than expected, so I had to order another set for my grandson. Nevertheless, I love the color, and they work wonderfully for me. /This blanket is fantastic! It's soft, cozy, and durable. While it's not overly heavy, it does a good job of keeping me comfortable in hot weather. I'll be buying another set for my daughter, who is having trouble sleeping. /I bought these curtains, and they looked pretty cute. They ar

In [167]:
test_dataset_raw_0_50['text_'][12791] = "This animated movie was fantastic! Initially, I didn't expect it to be as good as the original, but it pleasantly surprised me with its greatness."

<ipython-input-167-0e687f8424a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][12791] = "This animated movie was fantastic! Initially, I didn't expect it to be as good as the original, but it pleasantly surprised me with its greatness."


In [168]:
test_dataset_raw_0_50['text_'][21130] = "My cat absolutely loves this toy, and it keeps her entertained for a considerable amount of time. She doesn't chase it, but it still proves to be an excellent value for the price."

<ipython-input-168-54da43a17500>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][21130] = "My cat absolutely loves this toy, and it keeps her entertained for a considerable amount of time. She doesn't chase it, but it still proves to be an excellent value for the price."


In [169]:
test_dataset_raw_0_50['text_'][26333] = "Calling it 'broken' would be an understatement of the thrilling journey this story takes you on. Filled with captivating twists and turns, it keeps you hooked and eager for more. The characters are truly believable, and the fast-paced narrative adds to its appeal. Without a doubt, this book is a must-read, and I highly recommend it. I received this book from a friend, and in all honesty, I absolutely loved it!"

<ipython-input-169-efde937c0e47>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][26333] = "Calling it 'broken' would be an understatement of the thrilling journey this story takes you on. Filled with captivating twists and turns, it keeps you hooked and eager for more. The characters are truly believable, and the fast-paced narrative adds to its appeal. Without a doubt, this book is a must-read, and I highly recommend it. I received this book from a friend, and in all honesty, I absolutely loved it!"


In [170]:
test_dataset_raw_0_50['text_'][34131] = "This deck serves as an excellent starter set, featuring numerous high-quality cards. The variety among the cards is impressive, and the design is well-crafted."

<ipython-input-170-6cae279b0a14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][34131] = "This deck serves as an excellent starter set, featuring numerous high-quality cards. The variety among the cards is impressive, and the design is well-crafted."


In [171]:
test_dataset_raw_0_50['text_'][28036] ="Slight spoiler alert: Killian is portrayed as the male lead with a troubled past that has made him come across as a bit of a jerk. His desire to avoid involvement with the mercenary organization he was hired to protect was the only thing holding him back from making a decision. He had much to learn about the world and military, and it was a challenging lesson to grasp. Throughout the experience, he underwent significant personal growth, but it was hard to see him as anything other than a jerk.On the other hand, Kylo, a former Marine, is a young woman with a military background. Although she wasn't in a position to tell Killian how to behave, she was committed to protecting him. While she also had much to learn about her family and their opinions, she was determined to do everything in her power to keep him safe and make him happy. At the same time, Kylo felt that there was something about herself that would profoundly change her life for the better. The last thing she wanted was to..."

<ipython-input-171-37a1efa1aaa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][28036] ="Slight spoiler alert: Killian is portrayed as the male lead with a troubled past that has made him come across as a bit of a jerk. His desire to avoid involvement with the mercenary organization he was hired to protect was the only thing holding him back from making a decision. He had much to learn about the world and military, and it was a challenging lesson to grasp. Throughout the experience, he underwent significant personal growth, but it was hard to see him as anything other than a jerk.On the other hand, Kylo, a former Marine, is a young woman with a military background. Although she wasn't in a position to tell Killian how to behave, she was committed to protecting him. Whil

In [172]:
test_dataset_raw_0_50['text_'][21829] ="This product is fantastic for taking my two small dogs outside with me. I plan to keep using it for the next couple of months due to its great functionality and the excellent price. It works perfectly, especially since my dog is a chewer, and I find it easy to get her to eat with it."

<ipython-input-172-fe397863835c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][21829] ="This product is fantastic for taking my two small dogs outside with me. I plan to keep using it for the next couple of months due to its great functionality and the excellent price. It works perfectly, especially since my dog is a chewer, and I find it easy to get her to eat with it."


In [173]:
test_dataset_raw_0_50['text_'][36002] ="We bought this camera for my daughter, and she absolutely loves it, especially the bright colors. She plays with it frequently, and it's easy for her to use. The camera has been perfect for her as she enjoys taking pictures of everything she can, including her family's toys and other items. The camera's quality is impressive, and it's very durable and user-friendly. I highly recommend this camera for any child interested in video. It's an excellent choice, and I would recommend it to anyone who is interested."

<ipython-input-173-31b7ac0123c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][36002] ="We bought this camera for my daughter, and she absolutely loves it, especially the bright colors. She plays with it frequently, and it's easy for her to use. The camera has been perfect for her as she enjoys taking pictures of everything she can, including her family's toys and other items. The camera's quality is impressive, and it's very durable and user-friendly. I highly recommend this camera for any child interested in video. It's an excellent choice, and I would recommend it to anyone who is interested."


In [174]:
test_dataset_raw_0_50['text_'][32896] ="This item fits my 20-month-old perfectly and includes all the necessary pieces to make it function."

<ipython-input-174-9aaa60778100>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][32896] ="This item fits my 20-month-old perfectly and includes all the necessary pieces to make it function."


In [175]:
test_dataset_raw_0_50['text_'][13185] ="I haven't watched the entire movie yet, and I won't be purchasing the Blu-ray. Instead, I had to obtain a copy in some other way."

<ipython-input-175-c18ed5cfdc53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][13185] ="I haven't watched the entire movie yet, and I won't be purchasing the Blu-ray. Instead, I had to obtain a copy in some other way."


In [176]:
test_dataset_raw_0_50['text_'][4173] ="I purchased this as a gift for my husband, and he absolutely adores it. He loves the way it..."

<ipython-input-176-805f5e6dbdce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][4173] ="I purchased this as a gift for my husband, and he absolutely adores it. He loves the way it..."


In [177]:
test_dataset_raw_0_50['text_'][6265] ="While I typically shoot multiple weapons, I found it a bit challenging to acquire this particular one. However, I managed to get a solid grip on the gun, making it suitable for use with a Ruger 1911."

<ipython-input-177-755e6ebbb440>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][6265] ="While I typically shoot multiple weapons, I found it a bit challenging to acquire this particular one. However, I managed to get a solid grip on the gun, making it suitable for use with a Ruger 1911."


In [178]:
test_dataset_raw_0_50['text_'][36584] ="My granddaughter adores it. I purchased boxer briefs for my daughter, and she was thrilled with them."

<ipython-input-178-61f64eb8f375>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][36584] ="My granddaughter adores it. I purchased boxer briefs for my daughter, and she was thrilled with them."


In [179]:
test_dataset_raw_0_50['text_'][18300] ="I'm giving this product 4 stars because it's a cost-effective light that is suitable for various tasks.I previously purchased a similar light from Home Depot that was advertised as a 4-watt light, but it turned out to be a 5-watt light with no sun and a 2-watt dimmer. In contrast, the one I have now is genuinely a 4-watt light."

<ipython-input-179-a33e0ca4e81a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][18300] ="I'm giving this product 4 stars because it's a cost-effective light that is suitable for various tasks.I previously purchased a similar light from Home Depot that was advertised as a 4-watt light, but it turned out to be a 5-watt light with no sun and a 2-watt dimmer. In contrast, the one I have now is genuinely a 4-watt light."


In [180]:
test_dataset_raw_0_50['text_'][36500] ="I've always been a Star Wars fan, but I was pleasantly surprised by how well-made this particular item is. The attention to detail is fantastic, and the paintwork is exquisite. Despite my initial expectation of a smaller figure, I found it to be perfect in size. It's still enjoyable to play with and is a must-have for any Star Wars enthusiast.I bought this as a gift for my son, and he was genuinely interested in it. The toy is very cute, and he loved it. I'm looking forward to getting more items like this for him, knowing it will make him happy. Great product! My granddaughter adores these bath toys, but the kids prefer not to leave them outside for too long. I got them for my 9-year-old daughter on her birthday, and she loves playing with them, although she found the small pieces a bit challenging to handle. My granddaughter loved this doll, even though she's not generally into dolls. She was initially a little apprehensive about the wooden material, but she found the doll to be very cute. However, I felt it was a bit more expensive than expected, and the dress was too small and loose at the back. The dolls are also cheaply made and challenging to put on the doll's back. Despite these shortcomings, it's a fun toy overall."

<ipython-input-180-113611e17aff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][36500] ="I've always been a Star Wars fan, but I was pleasantly surprised by how well-made this particular item is. The attention to detail is fantastic, and the paintwork is exquisite. Despite my initial expectation of a smaller figure, I found it to be perfect in size. It's still enjoyable to play with and is a must-have for any Star Wars enthusiast.I bought this as a gift for my son, and he was genuinely interested in it. The toy is very cute, and he loved it. I'm looking forward to getting more items like this for him, knowing it will make him happy. Great product! My granddaughter adores these bath toys, but the kids prefer not to leave them outside for too long. I got them for my 9-yea

In [181]:
test_dataset_raw_0_50['text_'][8968] ="The product worked right away and significantly enhanced home security. The only downside is that it takes some time to get accustomed to using it."

<ipython-input-181-6e0491220eaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][8968] ="The product worked right away and significantly enhanced home security. The only downside is that it takes some time to get accustomed to using it."


In [182]:
test_dataset_raw_0_50['text_'][32790] ="This product works exceptionally well! The material is thick and durable, and the pieces are sturdy, capable of enduring the wear and tear of... "

<ipython-input-182-16121b8e7d9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][32790] ="This product works exceptionally well! The material is thick and durable, and the pieces are sturdy, capable of enduring the wear and tear of... "


In [183]:
test_dataset_raw_0_50['text_'][26048] ="I thoroughly enjoyed reading this P.J. Parrish book, and it was my first experience with this author's work. I loved it so much that I have already bought her other books, and I'm eagerly looking forward to reading them. I'm very excited about diving into more of her writing."

<ipython-input-183-b08172eb1bc9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][26048] ="I thoroughly enjoyed reading this P.J. Parrish book, and it was my first experience with this author's work. I loved it so much that I have already bought her other books, and I'm eagerly looking forward to reading them. I'm very excited about diving into more of her writing."


In [184]:
test_dataset_raw_0_50['text_'][10131] ="Adding an external HDD to your computer is incredibly easy with this product; all you have to do is plug it in. I had to replace a malfunctioning solid-state drive that wasn't compatible with the other drives I had. If you're considering making a purchase..."

<ipython-input-184-4b50a9810c73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][10131] ="Adding an external HDD to your computer is incredibly easy with this product; all you have to do is plug it in. I had to replace a malfunctioning solid-state drive that wasn't compatible with the other drives I had. If you're considering making a purchase..."


In [185]:
test_dataset_raw_0_50['text_'][28084] ="I thoroughly enjoy a good SEAL story, and this book didn't disappoint. The characters are well-developed and believable, and the writing is solid. I finished reading it in just a few days and found the plot to be strong, leading to a satisfying ending. The author also excels in describing the most common type of dog and its characteristics, making the book an enjoyable, quick, and easy read. I read the entire story in one sitting and found it captivating; I'm a fan of Ms. King's writing and look forward to more from her. Overall, it's a great start to a series, and I highly recommend this book, especially to those who appreciate westerns and seek an entertaining, quick read. The characters and plot are well-written, and the humor adds to the enjoyment of the story."

<ipython-input-185-ff259b867177>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][28084] ="I thoroughly enjoy a good SEAL story, and this book didn't disappoint. The characters are well-developed and believable, and the writing is solid. I finished reading it in just a few days and found the plot to be strong, leading to a satisfying ending. The author also excels in describing the most common type of dog and its characteristics, making the book an enjoyable, quick, and easy read. I read the entire story in one sitting and found it captivating; I'm a fan of Ms. King's writing and look forward to more from her. Overall, it's a great start to a series, and I highly recommend this book, especially to those who appreciate westerns and seek an entertaining, quick read. The cha

In [186]:
test_dataset_raw_0_50['text_'][6407] ="I am now interested in getting these in a larger size, as I have found them to be quite useful. I even used them for a few hours in a cave, and I am impressed with the solid seams and pleasant texture. So far, I haven't experienced any problems with the seams becoming loose."

<ipython-input-186-b72525e2818c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_50['text_'][6407] ="I am now interested in getting these in a larger size, as I have found them to be quite useful. I even used them for a few hours in a cave, and I am impressed with the solid seams and pleasant texture. So far, I haven't experienced any problems with the seams becoming loose."


In [187]:
merged_df_gpt = pd.concat([test_dataset_raw_0_50, test_dataset_raw_1_100], ignore_index=True)

# Shuffle the index
test_150_gpt = merged_df_gpt.sample(frac=1, random_state=42).reset_index(drop=True)

In [188]:
test_150_gpt

,category,rating,label,text_,target
0,Sports_and_Outdoors_5,5.0,OR,"be sure to check ser numbers for fit, mine loo...",1
1,Kindle_Store_5,5.0,CG,I derive immense pleasure from reading this se...,0
2,Sports_and_Outdoors_5,5.0,OR,I absolutely love my mat! I used to slip const...,1
3,Pet_Supplies_5,5.0,OR,Love this leash. GREAT training tool! Totall...,1
4,Pet_Supplies_5,4.0,OR,Apparently the fleas in my area (GA) are immun...,1
...,...,...,...,...,...
145,Books_5,5.0,OR,I wasn't expecting it to be a collection of st...,1
146,Movies_and_TV_5,4.0,OR,Boy Meets World as a show was great. I enjoyed...,1
147,Toys_and_Games_5,5.0,CG,This doll is absolutely adorable! She is incre...,0
148,Kindle_Store_5,4.0,OR,I enjoyed this book just shows you looking for...,1


In [189]:
test_encodings_gpt = tokenizer(test_150_gpt['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_gpt = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_gpt),
    test_150_gpt['target'].values
))


In [190]:
test_dataset_gpt_batched = test_dataset_gpt.batch(16)
y_pred_raw_gpt = model.predict(test_dataset_gpt_batched)

# Convert raw predictions to class predictions
y_pred_gpt = np.argmax(y_pred_raw_gpt.logits, axis=1)

# Get the actual class labels
y_true_gpt = []
for features, label in test_dataset_gpt:
    y_true_gpt.append(label.numpy())
y_true_gpt = np.array(y_true_gpt)


# Generate a classification report
report_gpt = classification_report(y_true_gpt, y_pred_gpt, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report_gpt)

10/10 [==============================] - 3s 252ms/step
              precision    recall  f1-score   support

          CG     0.9000    0.9000    0.9000        50
          OR     0.9500    0.9500    0.9500       100

    accuracy                         0.9333       150
   macro avg     0.9250    0.9250    0.9250       150
weighted avg     0.9333    0.9333    0.9333       150



# gpt 3.5 with 125

In [211]:
test_dataset_0_25 = test_dataset_raw_0_50.sample(n=25)
test_dataset_0_25_df = pd.DataFrame(test_dataset_0_25)

In [212]:
merged_df_gpt125 = pd.concat([test_dataset_0_25_df, test_dataset_raw_1_100], ignore_index=True)

# Shuffle the index
test_125_gpt = merged_df_gpt125.sample(frac=1, random_state=42).reset_index(drop=True)

In [213]:
test_encodings_gpt_125 = tokenizer(test_125_gpt['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_gpt_125 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_gpt_125),
    test_125_gpt['target'].values
))


In [214]:
test_dataset_gpt_batched_125 = test_dataset_gpt_125.batch(16)
y_pred_raw_gpt_125 = model.predict(test_dataset_gpt_batched_125)

# Convert raw predictions to class predictions
y_pred_gpt_125 = np.argmax(y_pred_raw_gpt_125.logits, axis=1)

# Get the actual class labels
y_true_gpt_125 = []
for features, label in test_dataset_gpt_125:
    y_true_gpt_125.append(label.numpy())
y_true_gpt_125 = np.array(y_true_gpt_125)


# Generate a classification report
report_gpt_125 = classification_report(y_true_gpt_125, y_pred_gpt_125, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report_gpt_125)

8/8 [==============================] - 2s 271ms/step
              precision    recall  f1-score   support

          CG     0.8276    0.9600    0.8889        25
          OR     0.9896    0.9500    0.9694       100

    accuracy                         0.9520       125
   macro avg     0.9086    0.9550    0.9291       125
weighted avg     0.9572    0.9520    0.9533       125



# chatgpt4

In [220]:
index_2 = [17408,  9913,  7820, 27662, 26833,  7804, 21469, 18647, 15587,
            39912, 17068,  2006, 22140, 33467, 22840, 35898, 30798, 25185,
            17279,  4867, 22505, 11525, 25079, 36858, 37468]

In [221]:
test_dataset_raw_0_25 = test_dataset_raw_0.loc[index_2]

In [217]:
random_25_rows = test_dataset_raw_0.sample(n=25)

# Build the new DataFrame using the randomly selected rows
test_dataset_raw_0_25 = pd.DataFrame(random_25_rows)

In [222]:
test_dataset_raw_0_25.index

Int64Index([17408,  9913,  7820, 27662, 26833,  7804, 21469, 18647, 15587,
            39912, 17068,  2006, 22140, 33467, 22840, 35898, 30798, 25185,
            17279,  4867, 22505, 11525, 25079, 36858, 37468],
           dtype='int64')

In [223]:
test_dataset_raw_0_25['text_'][37468]

'Fits true to size and looks great.  It is a little snug for my small size,'

In [224]:
test_dataset_raw_0_25['text_'][17408] ="Fantastic lights! Kudos to both the manufacturer and the seller! The quality is top-notch. It's a charming, compact light that functions perfectly. The pricing is also remarkably reasonable. The light unit itself is excellent and performs brilliantly."

<ipython-input-224-1799c2a950be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][17408] ="Fantastic lights! Kudos to both the manufacturer and the seller! The quality is top-notch. It's a charming, compact light that functions perfectly. The pricing is also remarkably reasonable. The light unit itself is excellent and performs brilliantly."


In [225]:
test_dataset_raw_0_25['text_'][9913] ="The item functions perfectly. However, the description doesn't include details about the shipping box. I'm planning to return this item. Despite this, it's an impressive product. I purchased this for a friend who is extremely satisfied with it. The only aspect I would like to change is..."

<ipython-input-225-079a0e5e2ba6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][9913] ="The item functions perfectly. However, the description doesn't include details about the shipping box. I'm planning to return this item. Despite this, it's an impressive product. I purchased this for a friend who is extremely satisfied with it. The only aspect I would like to change is..."


In [227]:
test_dataset_raw_0_25['text_'][7820] = "This is an essential item for anyone wanting to experiment with a new bat for the first time. I had a friend who was interested in a bat at a local store that was similar to one we'd used for batting practice. That bat performed exceptionally well, and we absolutely loved it. Despite being slightly on the pricier side, it met our requirements perfectly and was worth the investment. We recently went to Walmart and purchased a new bat. The cost was $30 less than the previous one, which delighted us. The bat was a tad heavy, so we stored it in a bag and placed it in a closet. Despite initial difficulties fitting it into our cooler, it ended up being a good fit and we used it for our batting practice. It's a superb bat, and we're pleased with our purchase. I would recommend it to anyone considering buying a bat. In addition, we acquired a tent for our boat journey along Mexico's east coast. It was astounding how warm and comfortable it was. The tent was not only spacious but also simple to set up. It even allowed us to sleep inside while we were out on the water, with a clear view of the water."

<ipython-input-227-2a2f11cbc04f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][7820] = "This is an essential item for anyone wanting to experiment with a new bat for the first time. I had a friend who was interested in a bat at a local store that was similar to one we'd used for batting practice. That bat performed exceptionally well, and we absolutely loved it. Despite being slightly on the pricier side, it met our requirements perfectly and was worth the investment. We recently went to Walmart and purchased a new bat. The cost was $30 less than the previous one, which delighted us. The bat was a tad heavy, so we stored it in a bag and placed it in a closet. Despite initial difficulties fitting it into our cooler, it ended up being a good fit and we used it for our ba

In [228]:
test_dataset_raw_0_25['text_'][27662] = "I found the book incredibly captivating. The characters were skillfully fleshed out, making the storyline quite enjoyable. I received an advance reader's copy in return for an unbiased review. This book is a gem; the characters and plot are beautifully crafted. It serves as an excellent series for the younger audience. Having read the initial book in the series, I can vouch for the overall quality of the series. I strongly recommend this book. This book was a pleasure to read. I am a fan of good romance, and this had a hint of mystery, enhancing the overall story. This book was appealing, though initially, there was a bit of confusion. However, once I overcame the initial ambiguity, the read was worthwhile. I'm eagerly anticipating that the upcoming books in this series will follow suit. This charming tale revolves around a young man and a girl named Belinda who..."

<ipython-input-228-9e3fc059b1d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][27662] = "I found the book incredibly captivating. The characters were skillfully fleshed out, making the storyline quite enjoyable. I received an advance reader's copy in return for an unbiased review. This book is a gem; the characters and plot are beautifully crafted. It serves as an excellent series for the younger audience. Having read the initial book in the series, I can vouch for the overall quality of the series. I strongly recommend this book. This book was a pleasure to read. I am a fan of good romance, and this had a hint of mystery, enhancing the overall story. This book was appealing, though initially, there was a bit of confusion. However, once I overcame the initial ambiguity

In [229]:
test_dataset_raw_0_25['text_'][26833] = "This is the sequel in The Crystal Queen series. The Crystal Queen series has held a special place in my reading list for the past few years, and I'm confident that I'll continue to explore more of her works in the future. This book was an absolute delight! A friend gifted me this book in return for a candid review. I found it marvelous, brimming with compelling characters and unexpected plot twists. I heartily endorse it! It was..."

<ipython-input-229-3c68ea8e946e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][26833] = "This is the sequel in The Crystal Queen series. The Crystal Queen series has held a special place in my reading list for the past few years, and I'm confident that I'll continue to explore more of her works in the future. This book was an absolute delight! A friend gifted me this book in return for a candid review. I found it marvelous, brimming with compelling characters and unexpected plot twists. I heartily endorse it! It was..."


In [230]:
test_dataset_raw_0_25['text_'][7804] ="The only reason I rated this a 3-star is because I didn't want to unduly skew the ratings. It may not be the best in terms of design, but it's undoubtedly a superior product. I also gave it 4 stars at one point, not because I desired to use it, but rather I needed it to kickstart my day. Despite its effectiveness, I didn't intend to use it for anything else. My curiosity was mainly around whether it could perform that specific task, and it did. However, it's challenging to always ascertain if you're making a mistake. It's a remarkable product, and I plan to purchase more of it. I'll certainly retain it for my wife. Regarding the tent, I've been using it for several years without encountering any issues. I employed this tent during a boat ride in the summer of 2012, and its durability has been impressive. The tent boasts a substantial footprint and appears to be well-constructed. I'd recommend it to anyone looking to establish a tent on a boat. I've used this tent on a few other boats..."

<ipython-input-230-bba4c5859c4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][7804] ="The only reason I rated this a 3-star is because I didn't want to unduly skew the ratings. It may not be the best in terms of design, but it's undoubtedly a superior product. I also gave it 4 stars at one point, not because I desired to use it, but rather I needed it to kickstart my day. Despite its effectiveness, I didn't intend to use it for anything else. My curiosity was mainly around whether it could perform that specific task, and it did. However, it's challenging to always ascertain if you're making a mistake. It's a remarkable product, and I plan to purchase more of it. I'll certainly retain it for my wife. Regarding the tent, I've been using it for several years without enco

In [231]:
test_dataset_raw_0_25['text_'][21469] ="This product shields my kittens from the odor of the litter box. Additionally, I appreciate that it's one of the initial items I've purchased for my kitty that we're not constantly having to remove and..."

<ipython-input-231-f94a80caef7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][21469] ="This product shields my kittens from the odor of the litter box. Additionally, I appreciate that it's one of the initial items I've purchased for my kitty that we're not constantly having to remove and..."


In [232]:
test_dataset_raw_0_25['text_'][18647] ="The Bosch template adapters can be quite a challenge to locate. I find myself needing to switch them off and on. I replaced the original adapters with these ones, which was a surprisingly straightforward task. They continue to function excellently, leaving me very satisfied. I purchased this item to replace a damaged one I previously owned. I appreciate the ability to use the hose to clear the ballast from my sink. After a few installations, it has proven to be the ideal replacement for the broken one. I'm now planning to..."

<ipython-input-232-463af6542b10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][18647] ="The Bosch template adapters can be quite a challenge to locate. I find myself needing to switch them off and on. I replaced the original adapters with these ones, which was a surprisingly straightforward task. They continue to function excellently, leaving me very satisfied. I purchased this item to replace a damaged one I previously owned. I appreciate the ability to use the hose to clear the ballast from my sink. After a few installations, it has proven to be the ideal replacement for the broken one. I'm now planning to..."


In [233]:
test_dataset_raw_0_25['text_'][15587] ="Fantastic tool from an impressive toolbox. The only issue is that it includes a miniature screwdriver."

<ipython-input-233-1627f4fb74c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][15587] ="Fantastic tool from an impressive toolbox. The only issue is that it includes a miniature screwdriver."


In [234]:
test_dataset_raw_0_25['text_'][39912] ="I am the proud owner of an OMAPC, and the accompanying knee pads are a perfect fit. The heel size is just right, and the material is pleasantly soft and comfy. However, I encountered an issue with the material being excessively loose. Consequently, I'll be returning it. I opted for a size up on reorder, and it fitted perfectly. I had to wash the boots on the first day, but they're incredibly comfortable. I've yet to wear them for an extended period, but finding boots that remain comfortable for long durations has always been a challenge. For reference, I'm 5'4 and weigh 140lbs, and these..."

<ipython-input-234-14adfe5550da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][39912] ="I am the proud owner of an OMAPC, and the accompanying knee pads are a perfect fit. The heel size is just right, and the material is pleasantly soft and comfy. However, I encountered an issue with the material being excessively loose. Consequently, I'll be returning it. I opted for a size up on reorder, and it fitted perfectly. I had to wash the boots on the first day, but they're incredibly comfortable. I've yet to wear them for an extended period, but finding boots that remain comfortable for long durations has always been a challenge. For reference, I'm 5'4 and weigh 140lbs, and these..."


In [235]:
test_dataset_raw_0_25['text_'][17068] ="The quality is appalling. Do not squander valuable metal or steel on it. The sole reason I purchased this item was to replace a faulty..."

<ipython-input-235-65c5ce737675>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][17068] ="The quality is appalling. Do not squander valuable metal or steel on it. The sole reason I purchased this item was to replace a faulty..."


In [236]:
test_dataset_raw_0_25['text_'][2006] ="Despite its modest size, this machine makes a sound investment. The only drawback is its compact size. The components needed to attach the cover at the top were..."

<ipython-input-236-a98989124ecc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][2006] ="Despite its modest size, this machine makes a sound investment. The only drawback is its compact size. The components needed to attach the cover at the top were..."


In [237]:
test_dataset_raw_0_25['text_'][22140] ="I own two small breed dogs and felt this would be a beneficial enhancement for the litter box. The top comes in a striking bright color and is simple to clean. I used this tool to tidy up my older cat's litter box, although it didn't include a large filter."

<ipython-input-237-0a2258be8c09>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][22140] ="I own two small breed dogs and felt this would be a beneficial enhancement for the litter box. The top comes in a striking bright color and is simple to clean. I used this tool to tidy up my older cat's litter box, although it didn't include a large filter."


In [238]:
test_dataset_raw_0_25['text_'][33467] ="Delightfully soft and plush. My daughter adores it, and her two-year-old sister shares the same sentiment."

<ipython-input-238-50f3d8653dff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][33467] ="Delightfully soft and plush. My daughter adores it, and her two-year-old sister shares the same sentiment."


In [239]:
test_dataset_raw_0_25['text_'][22840] ="Both my Aussie and my African Grey are fond of these! I plan to keep them stocked for the next few months. This food is a big hit with my dogs. My dog consumes it leisurely, first from a deep, slow-feed bowl, then from a larger bowl. I own two dogs, each weighing 4 lbs., and one of them is quite finicky about food. I've purchased a second bag to use for her vet visits. The two bags I've received so far are strikingly similar."

<ipython-input-239-8001721759f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][22840] ="Both my Aussie and my African Grey are fond of these! I plan to keep them stocked for the next few months. This food is a big hit with my dogs. My dog consumes it leisurely, first from a deep, slow-feed bowl, then from a larger bowl. I own two dogs, each weighing 4 lbs., and one of them is quite finicky about food. I've purchased a second bag to use for her vet visits. The two bags I've received so far are strikingly similar."


In [240]:
test_dataset_raw_0_25['text_'][35898] ="I was taken aback by the level of detail in the figures. The Batman figure is notably impressive, although the figure itself is somewhat diminutive, and the cover seems quite fragile. I wouldn't advise anyone to purchase this figure. Nonetheless, it remains an entertaining toy to play with and makes a fine addition to any collection. This product is exceptionally well-crafted. My daughter has engaged with it for approximately two months now and continues to enjoy it immensely. I'm extremely satisfied with this purchase. My son is nearly 3, and..."

<ipython-input-240-57a7ee43906d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][35898] ="I was taken aback by the level of detail in the figures. The Batman figure is notably impressive, although the figure itself is somewhat diminutive, and the cover seems quite fragile. I wouldn't advise anyone to purchase this figure. Nonetheless, it remains an entertaining toy to play with and makes a fine addition to any collection. This product is exceptionally well-crafted. My daughter has engaged with it for approximately two months now and continues to enjoy it immensely. I'm extremely satisfied with this purchase. My son is nearly 3, and..."


In [241]:
test_dataset_raw_0_25['text_'][30798] ="I went through the whole book, and it proved to be a fascinating read. The narrative commences when Kary, one of the two girls, is assaulted by an enigmatic character. The abductor is portrayed as a devilish figure. Interestingly, Kary's father held the position of a priest, and strikingly, so did the kidnapper."

<ipython-input-241-2b0cad0f7085>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][30798] ="I went through the whole book, and it proved to be a fascinating read. The narrative commences when Kary, one of the two girls, is assaulted by an enigmatic character. The abductor is portrayed as a devilish figure. Interestingly, Kary's father held the position of a priest, and strikingly, so did the kidnapper."


In [242]:
test_dataset_raw_0_25['text_'][25185] ="This author is a new discovery for me. I found her work captivating and am eager to delve into more of her books. I received an advance reader's copy in return for my candid review. I'm smitten with this book."

<ipython-input-242-679f1a5903dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][25185] ="This author is a new discovery for me. I found her work captivating and am eager to delve into more of her books. I received an advance reader's copy in return for my candid review. I'm smitten with this book."


In [243]:
test_dataset_raw_0_25['text_'][17279] = "These are indeed very impressive - a high-quality tool set. I plan to retain them for the next few years. They were a bargain at a fantastic price! They function excellently. The light is somewhat brighter..."

<ipython-input-243-97b3311a55c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][17279] = "These are indeed very impressive - a high-quality tool set. I plan to retain them for the next few years. They were a bargain at a fantastic price! They function excellently. The light is somewhat brighter..."


In [244]:
test_dataset_raw_0_25['text_'][4867] ="This clock is a must-have! I absolutely adore the texture of it."

<ipython-input-244-b606dcfa57a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][4867] ="This clock is a must-have! I absolutely adore the texture of it."


In [245]:
test_dataset_raw_0_25['text_'][22505] ="I'm a big fan of this kit. My puppy, being an intense chewer, makes quick work of it, and I love how quickly I can toss this thing. I've been using it along with the Tasers on a 4lb bag of dog food, and she doesn't object to it at all. I keep another 4 lb bag of dog food handy, which is excellent for her to carry around in her mouth. She's absolutely wild about this."

<ipython-input-245-0bced201bf32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][22505] ="I'm a big fan of this kit. My puppy, being an intense chewer, makes quick work of it, and I love how quickly I can toss this thing. I've been using it along with the Tasers on a 4lb bag of dog food, and she doesn't object to it at all. I keep another 4 lb bag of dog food handy, which is excellent for her to carry around in her mouth. She's absolutely wild about this."


In [246]:
test_dataset_raw_0_25['text_'][11525] ="The arm functions excellently. I use it as an extended cable for my computer and also have one in my bedroom. The cable is robust and easy to operate. I would certainly recommend it as a valuable addition to your computer setup. I acquired this device with the intention of utilizing it as a family-friendly DVD player. I own a Samsung Galaxy S4 and a Sony PS3. The Sony is the only device among them that I haven't tested yet. I wanted to make my Galaxy S4 compatible with my Samsung, but I was concerned that a lengthy cord might reduce its lifespan. I came across the Sony on a sale at Best Buy and decided to give it a shot. It doesn't feature a cord but does offer a USB port, which is incredibly useful for transferring DVD files."

<ipython-input-246-6af7de42c6e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][11525] ="The arm functions excellently. I use it as an extended cable for my computer and also have one in my bedroom. The cable is robust and easy to operate. I would certainly recommend it as a valuable addition to your computer setup. I acquired this device with the intention of utilizing it as a family-friendly DVD player. I own a Samsung Galaxy S4 and a Sony PS3. The Sony is the only device among them that I haven't tested yet. I wanted to make my Galaxy S4 compatible with my Samsung, but I was concerned that a lengthy cord might reduce its lifespan. I came across the Sony on a sale at Best Buy and decided to give it a shot. It doesn't feature a cord but does offer a USB port, which is 

In [250]:
test_dataset_raw_0_25['text_'][25079] ="I'm uncertain about the origins of some individuals, but comprehending the events that transpired is crucial.I found considerable enjoyment in reading this book and I..."

<ipython-input-250-5a87d1bbc00d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][25079] ="I'm uncertain about the origins of some individuals, but comprehending the events that transpired is crucial.I found considerable enjoyment in reading this book and I..."


In [251]:
test_dataset_raw_0_25['text_'][36858] ="It has an odd fit, tight around the thighs. I'm unsure if the knee section is considerably tighter, but..."

<ipython-input-251-29c34c3434ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][36858] ="It has an odd fit, tight around the thighs. I'm unsure if the knee section is considerably tighter, but..."


In [253]:
test_dataset_raw_0_25['text_'][37468] ="It fits accurately to size and has an appealing appearance. However, for my petite frame, it's slightly tight."

<ipython-input-253-b8b729cc3458>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset_raw_0_25['text_'][37468] ="It fits accurately to size and has an appealing appearance. However, for my petite frame, it's slightly tight."


In [254]:
merged_df_120 = pd.concat([test_dataset_raw_0_25, test_dataset_raw_1_100], ignore_index=True)

# Shuffle the index
test_120 = merged_df_120.sample(frac=1, random_state=42).reset_index(drop=True)

In [255]:
test_encodings_120 = tokenizer(test_120['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_120 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_120),
    test_120['target'].values
))

In [256]:
test_dataset_120_batched = test_dataset_120.batch(16)
y_pred_raw_120 = model.predict(test_dataset_120_batched)

# Convert raw predictions to class predictions
y_pred_120 = np.argmax(y_pred_raw_120.logits, axis=1)

# Get the actual class labels
y_true_120 = []
for features, label in test_dataset_120:
    y_true_120.append(label.numpy())
y_true_120 = np.array(y_true_120)


# Generate a classification report
report_120 = classification_report(y_true_120, y_pred_120, target_names=["CG", "OR"],digits =4)  # replace target_names with your actual class names
print(report_120)

8/8 [==============================] - 2s 274ms/step
              precision    recall  f1-score   support

          CG     0.8333    1.0000    0.9091        25
          OR     1.0000    0.9500    0.9744       100

    accuracy                         0.9600       125
   macro avg     0.9167    0.9750    0.9417       125
weighted avg     0.9667    0.9600    0.9613       125



# GPT4 random 25

In [257]:
# Dictionary with your texts and target values
data = {
    "text_": [
        "This kitchen blender is not only a perfect companion for creating delightful smoothies but also an elegant addition to any kitchen countertop.",
        "The comfort and sound quality of these headphones are nothing short of impressive. The noise cancellation feature allows for an immersive experience.",
        "This book is an absolute page-turner. The storyline is intriguing, the characters are well-developed, and the author's writing style is compelling.",
        "I was thoroughly impressed with this smartphone. It offers excellent performance, a crisp display, and a battery that lasts all day.",
        "This cookware set exceeded my expectations. It's not only aesthetically pleasing but also has an amazing non-stick surface and heats evenly.",
        "These running shoes are fantastic. They offer great support, are lightweight, and incredibly comfortable.",
        "The soundbar provides clear, room-filling sound. The setup was straightforward, and the Bluetooth connectivity is a great feature.",
        "This backpack is not only stylish but also functional. It has a generous storage capacity, and the material is durable.",
        "The vacuum cleaner is extremely efficient. The powerful suction is a plus, and it maneuvers around furniture effortlessly.",
        "The camping tent is easy to set up and spacious. The material is durable, withstanding various weather conditions.",
        "This pet bed is a hit with my dog. It's plush and comfortable, providing a cozy spot for her to rest.",
        "The keyboard delivers an enjoyable typing experience. The keys are responsive and comfortable to use for prolonged periods.",
        "This skincare product has transformed my skin. It's gentle, yet effective, leaving my skin feeling soft, refreshed, and visibly brighter.",
        "These wireless earbuds have an impressive sound quality, a comfortable fit, and the battery life is long-lasting.",
        "This yoga mat is excellent. It provides enough cushioning, has a non-slip surface, and is easy to clean.",
        "I love this coffee maker. It brews coffee quickly, it's easy to use, and the coffee tastes great.",
        "The slow cooker is a gem. It's easy to operate, the food comes out delicious, and it's easy to clean.",
        "This facial cleanser is wonderful. It cleanses deeply, leaving the skin feeling fresh and clean without drying it out.",
        "The fitness tracker is an excellent workout companion. It's accurate, easy to use, and the app provides insightful data.",
        "This bedsheet set is fantastic. It's incredibly soft, comfortable, and the color hasn't faded after several washes.",
        "This nail polish is amazing. The color is vibrant, it applies smoothly, and it has a long-lasting finish.",
        "This computer mouse is reliable and comfortable to use. It moves smoothly and the click is quiet.",
        "The baby monitor gives peace of mind. The video is clear, the audio is crisp, and the range is good.",
        "This electric toothbrush cleans teeth thoroughly. It has different modes for different needs, and the battery life is good.",
        "This lawn mower is a workhorse. It's powerful, easy to maneuver, and the cut is even."
    ],
    "target": [0] * 25
}

# Create DataFrame
test_dataset_raw_r_25 = pd.DataFrame(data)

In [258]:
test_dataset_raw_2_100 = test_dataset_raw_1_100[['text_','target']]

In [259]:
merged_df_120_2 = pd.concat([test_dataset_raw_r_25, test_dataset_raw_2_100], ignore_index=True)

# Shuffle the index
test_120_2 = merged_df_120_2.sample(frac=1, random_state=42).reset_index(drop=True)

In [260]:
test_encodings_120_2 = tokenizer(test_120_2['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_120_2 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_120_2),
    test_120_2['target'].values
))

In [261]:
test_dataset_120_2_batched = test_dataset_120_2.batch(16)
y_pred_raw_120_2 = model.predict(test_dataset_120_2_batched)

# Convert raw predictions to class predictions
y_pred_120_2 = np.argmax(y_pred_raw_120_2.logits, axis=1)

# Get the actual class labels
y_true_120_2 = []
for features, label in test_dataset_120_2:
    y_true_120_2.append(label.numpy())
y_true_120_2 = np.array(y_true_120_2)


# Generate a classification report
report_120_2 = classification_report(y_true_120_2, y_pred_120_2, target_names=["CG", "OR"],digits =4)
print(report_120_2)

8/8 [==============================] - 2s 267ms/step
              precision    recall  f1-score   support

          CG     0.8333    1.0000    0.9091        25
          OR     1.0000    0.9500    0.9744       100

    accuracy                         0.9600       125
   macro avg     0.9167    0.9750    0.9417       125
weighted avg     0.9667    0.9600    0.9613       125



# GPT 4 50

In [262]:
data1 ={
    "text_": [
        "This kitchen blender is not only a perfect companion for creating delightful smoothies but also an elegant addition to any kitchen countertop.",
        "The comfort and sound quality of these headphones are nothing short of impressive. The noise cancellation feature allows for an immersive experience.",
        "This book is an absolute page-turner. The storyline is intriguing, the characters are well-developed, and the author's writing style is compelling.",
        "I was thoroughly impressed with this smartphone. It offers excellent performance, a crisp display, and a battery that lasts all day.",
        "This cookware set exceeded my expectations. It's not only aesthetically pleasing but also has an amazing non-stick surface and heats evenly.",
        "These running shoes are fantastic. They offer great support, are lightweight, and incredibly comfortable.",
        "The soundbar provides clear, room-filling sound. The setup was straightforward, and the Bluetooth connectivity is a great feature.",
        "This backpack is not only stylish but also functional. It has a generous storage capacity, and the material is durable.",
        "The vacuum cleaner is extremely efficient. The powerful suction is a plus, and it maneuvers around furniture effortlessly.",
        "The camping tent is easy to set up and spacious. The material is durable, withstanding various weather conditions.",
        "This pet bed is a hit with my dog. It's plush and comfortable, providing a cozy spot for her to rest.",
        "The keyboard delivers an enjoyable typing experience. The keys are responsive and comfortable to use for prolonged periods.",
        "This skincare product has transformed my skin. It's gentle, yet effective, leaving my skin feeling soft, refreshed, and visibly brighter.",
        "These wireless earbuds have an impressive sound quality, a comfortable fit, and the battery life is long-lasting.",
        "This yoga mat is excellent. It provides enough cushioning, has a non-slip surface, and is easy to clean.",
        "I love this coffee maker. It brews coffee quickly, it's easy to use, and the coffee tastes great.",
        "The slow cooker is a gem. It's easy to operate, the food comes out delicious, and it's easy to clean.",
        "This facial cleanser is wonderful. It cleanses deeply, leaving the skin feeling fresh and clean without drying it out.",
        "The fitness tracker is an excellent workout companion. It's accurate, easy to use, and the app provides insightful data.",
        "This bedsheet set is fantastic. It's incredibly soft, comfortable, and the color hasn't faded after several washes.",
        "This nail polish is amazing. The color is vibrant, it applies smoothly, and it has a long-lasting finish.",
        "This computer mouse is reliable and comfortable to use. It moves smoothly and the click is quiet.",
        "The baby monitor gives peace of mind. The video is clear, the audio is crisp, and the range is good.",
        "This electric toothbrush cleans teeth thoroughly. It has different modes for different needs, and the battery life is good.",
        "This lawn mower is a workhorse. It's powerful, easy to maneuver, and the cut is even.",
        "The Zephyr earbuds offer a crystal clear sound experience. They fit snugly and stay in place, even during my most intense workouts. The battery life is impressive, and the touch controls are intuitive. Highly recommend!",
        "I've tried countless night creams, but LunaLuxe is by far my favorite. Within a week, I noticed a softer, more hydrated complexion. Plus, the light lavender scent is such a calming bonus before bedtime.",
        "Took these on a 10-mile hike right out of the box, and not a single blister! The TrekMaster boots provide great support and traction. They're also surprisingly lightweight. A must-have for any hiking enthusiast.",
        "I never thought I'd be raving about a blender, but here we are. The StellarBlend is powerful, easy to clean, and makes the creamiest smoothies. The preset modes are a game-changer. Worth every penny!",
        "PuraPure straws are not only sustainable but also durable and stylish. They come in an array of colors, and I love that they're dishwasher safe. Doing my bit for the environment just became a tad cooler.",
        "Dropped my phone down a flight of stairs – it was a heart-stopping moment. But thanks to the TechGuard case, my phone didn't have a scratch! It's slim, not bulky, and offers top-notch protection.",
        "I've been a die-hard physical book fan, but the BrioBook has converted me. The e-ink display is easy on the eyes, and the battery lasts forever. The built-in backlight is perfect for late-night reading.",
        "Apart from being a stylish timepiece, the SolarWave's solar charging feature is genuinely impressive. I've not had to worry about the battery at all. The minimalist design and eco-friendly feature make it a conversation starter.",
        "A quick spritz of AquaSpritz is all I need to feel refreshed. It's a lifesaver, especially during long flights or hot days. The mist is fine and doesn't disturb my makeup. Plus, the rosewater scent is divine.",
        "The GlideFlow yoga mat provides just the right amount of cushion and grip. I've tried many mats, but this one feels the most stable, especially during challenging poses. And the alignment lines? Genius!",
"This lamp is a tech lover's dream! Not only does it offer adjustable brightness settings, but it also syncs with my smart home system flawlessly. Plus, the modern design is the cherry on top.",
        "Switching to EcoLife wraps has drastically reduced my plastic usage. They're easy to clean, seal well, and come in adorable patterns. I feel good knowing I'm making a sustainable choice.",
        "Every time I wear Mystique, I receive endless compliments. It's a unique blend of floral and musk, lasting all day without being overpowering. A little goes a long way!",
        "The fabric is soft yet durable, wicking away sweat during intense workouts. ProFit leggings don't roll down, and the pocket feature is super handy. Five stars!",
        "From my phone to a physical photo in minutes! PixelPrint delivers vibrant, high-quality prints. The companion app's editing tools are a fantastic touch. No regrets on this purchase.",
        "Cooking has never been better! This pan heats evenly, is easy to clean, and feels built to last a lifetime. Chef's Delight has truly lived up to its name.",
        "Set up was a breeze, and it withstood a heavy downpour with no leaks. SkyHigh's tent is spacious and packed with thoughtful features. Camping trips just got an upgrade!",
        "No more stiff necks on flights! The AeroFlow pillow provides perfect support and is compact enough to fit in my carry-on. I won't travel without it.",
        "As an artist, the Novelty tablet has enhanced my digital creations. The pen feels natural, the screen is responsive, and the software bundle is generous. A masterpiece in its own right!",
        "My hair has never felt or looked better! LushLocks serum tames frizz and adds a beautiful shine without feeling greasy. A staple in my haircare routine.",
        "Sleeping on clouds! That's how the BellaCasa bedding feels. The fabric is soft yet breathable, and the patterns are elegant. Worth every penny.",
        "For a beginner gardener, this kit is perfect. GreenThumb provides everything I need, from seeds to tools. My garden is thriving thanks to their quality products.",
        "A mesmerizing addition to my backyard! The SolarZen lights charge quickly, glow beautifully, and have held up against all weather. No more tangled extension cords!",
        "Sleep has never been this rejuvenating. The DriftAway mattress contours to my body, providing the right balance of support and comfort. Plus, the cooling technology is a game-changer.",

"Tap water has never tasted so fresh! The EverPure filter was easy to install, and I've noticed a significant difference in water clarity and taste. A must-have for every household."





    ],
    "target": [0] * 50
}


In [263]:
test_dataset_raw_r_50 = pd.DataFrame(data1)

In [264]:
test_dataset_raw_3_100 = test_dataset_raw_1_100[['text_','target']]

In [265]:
merged_df_120_3 = pd.concat([test_dataset_raw_r_50, test_dataset_raw_3_100], ignore_index=True)

# Shuffle the index
test_120_3 = merged_df_120_3.sample(frac=1, random_state=42).reset_index(drop=True)

In [266]:
test_encodings_120_3 = tokenizer(test_120_3['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_120_3 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_120_3),
    test_120_3['target'].values
))

In [267]:
test_dataset_120_3_batched = test_dataset_120_3.batch(16)
y_pred_raw_120_3 = model.predict(test_dataset_120_3_batched)

# Convert raw predictions to class predictions
y_pred_120_3 = np.argmax(y_pred_raw_120_3.logits, axis=1)

# Get the actual class labels
y_true_120_3 = []
for features, label in test_dataset_120_3:
    y_true_120_3.append(label.numpy())
y_true_120_3 = np.array(y_true_120_3)


# Generate a classification report
report_120_3 = classification_report(y_true_120_3, y_pred_120_3, target_names=["CG", "OR"],digits =4)
print(report_120_3)

10/10 [==============================] - 3s 264ms/step
              precision    recall  f1-score   support

          CG     0.8889    0.8000    0.8421        50
          OR     0.9048    0.9500    0.9268       100

    accuracy                         0.9000       150
   macro avg     0.8968    0.8750    0.8845       150
weighted avg     0.8995    0.9000    0.8986       150



# GPT 3 50

In [268]:
data2 ={
    "text_": [
        "These earbuds are amazing! The sound quality is top-notch and the wireless connectivity is seamless. I love how comfortable they are in my ears, making them perfect for long listening sessions.",
        "This portable charger is a lifesaver. It's compact and lightweight, making it easy to carry around. It charges my phone quickly and has saved me from running out of battery multiple times.",
        "I'm loving this fitness tracker! It accurately monitors my steps, heart rate, and sleep patterns. The app is user-friendly and helps me stay on top of my health goals.",
        "Cooking has become a breeze with this non-stick pan. It heats up evenly and the food slides right off. Cleanup is a breeze, too! Definitely a must-have in the kitchen.",
"I take this water bottle everywhere I go. It keeps my water cold for hours, and I love that it's eco-friendly. The design is sleek and the quality is excellent.",
        "These headphones are a game-changer. The noise cancellation is impressive, whether I'm on a noisy plane or just want to focus at home. The sound quality is fantastic as well.",
        "I can't imagine my life without this smart home assistant. It controls my smart devices, plays my favorite music, and even answers my questions. It's like having a personal assistant at home.",
        "Say goodbye to pre-ground coffee! This coffee maker with a built-in grinder takes freshness to a new level. Every cup is a delight, and the aroma is heavenly.",
        "These suitcases exceeded my expectations. They're sturdy, well-built, and the wheels glide smoothly. The different sizes are perfect for various travel needs.",
        "I've been using these resistance bands for my home workouts, and they add the perfect level of challenge. The set comes with different resistance levels, making it versatile.",
        "Reading has never been this convenient. The e-reader tablet's screen resembles real paper, and I can carry a whole library with me. Battery life is impressive, too.",
        "Finally, no more tangled wires on my desk. This wireless mouse is responsive and comfortable to use. It's a small change that has made a big difference.",
        "As a yoga enthusiast, I appreciate the alignment lines on this mat. It helps me maintain proper positioning during poses. The thickness is just right for cushioning.",
        "I'm amazed by how crispy and delicious the food turns out in this air fryer. It's a healthier alternative to frying, and I use it almost every day.",
        "These headphones stay in place no matter how intense my workouts get. The sound quality is impressive, and they're sweat-resistant, which is a huge plus.",
        "This video doorbell adds a new level of security to my home. I can see and talk to visitors from my phone, even when I'm not home. Installation was straightforward.",
        "My teeth have never felt cleaner! This electric toothbrush not only cleans effectively but also has a UV sanitizer to keep the brush head germ-free. A great investment in oral hygiene.",
        "Cleaning has become so much easier with this handheld steam cleaner. It tackles grime and dirt effortlessly, and I love that it uses only water for a chemical-free clean.",
        "I no longer have to hunch over my laptop. This stand is adjustable and elevates my screen to eye level, saving my neck and improving my posture.",
        "This Bluetooth speaker is a party in itself! The LED lights sync with the music and create a fun atmosphere. The sound quality is great, making it perfect for gatherings.",
"I'm amazed by the versatility of this food processor. It slices, dices, and even kneads dough with ease. It has definitely become an essential tool in my kitchen.",
        "This speaker delivers impressive sound quality for its size. The Bluetooth connection is stable, and the battery life is excellent. I take it with me everywhere.",
        "Switching between sitting and standing is a game-changer for my productivity. This desk converter is easy to adjust and provides ample space for my monitors and keyboard.",
        "These plant pots have taken the guesswork out of watering my plants. The self-watering system is effective, and my plants have never looked healthier.",
        "These lights have added charm to my garden. They charge during the day and illuminate my outdoor space beautifully at night. Plus, they're eco-friendly!",
        "Capturing breathtaking aerial shots has never been this accessible. This foldable drone is easy to control and the camera quality exceeds my expectations.",
"Cleaning has become less of a chore with this cordless vacuum. It's lightweight, powerful, and the battery lasts long enough to clean my whole house.",
"These knives have greatly improved my kitchen skills. They're sharp, well-balanced, and the variety of sizes makes meal prep a breeze.",
        "This foam roller is a lifesaver for post-workout recovery. It helps relieve muscle tension and the textured surface adds an extra level of relief.",
        "As an artist, this drawing tablet has opened up new creative possibilities for me. The pressure sensitivity and precision are impressive.",
        "Controlling my home's temperature remotely is so convenient. This smart thermostat has helped me save on energy bills while keeping my space comfortable.",
        "This tumbler keeps my drinks hot or cold for hours. The stainless steel design is durable, and it's a great alternative to disposable cups.",
        "I've gained valuable insights into my sleep patterns with this smartwatch. It's comfortable to wear, and the app provides detailed sleep analysis.",
        "This folding bike is perfect for urban commuting. It's lightweight, easy to fold, and the ride is surprisingly smooth.",
        "Charging my devices is hassle-free with this wireless charging pad. It works seamlessly and eliminates the need for tangled cables.",
        "This sanitizing wand is a must-have for keeping my belongings clean. It's portable and uses UV-C light to disinfect surfaces effectively.",
        "Opening wine bottles has never been easier. This electric wine opener is quick, efficient, and adds a touch of elegance to wine nights.",
        "These food containers are a space-saving marvel. They collapse for easy storage and are perfect for packing lunches and storing leftovers.",
        "This keyboard is a game-changer for typing on my tablet or phone. It's compact, connects seamlessly via Bluetooth, and the keys have a satisfying feel.",
        "I'm amazed by how quickly I can whip up meals with this pressure cooker. It's versatile, safe, and the flavors of my dishes are intensified.",
        "This charging case keeps my earbuds powered up on the go. It's compact and convenient, ensuring I never run out of battery.",
        "Say goodbye to fumbling with your phone in the car. This magnetic mount holds my phone securely, and the installation is a breeze.",
        "These adjustable dumbbells have replaced a whole rack in my home gym. They're easy to change, and the weight range is perfect for various exercises.",
        "I'm enjoying café-quality espresso at home with this compact machine. It's easy to use, and the crema on my shots is impressive.",
        "Movie nights have become more exciting with this mini projector. The image quality is surprisingly good, and it's great for outdoor gatherings.",
        "This razor gives me a close shave without irritation. The battery life is impressive, and the design makes it comfortable to hold.",
        "Managing multiple devices is a breeze with this charging station. It keeps everything organized and charges devices quickly.",
"This protein powder is a tasty addition to my post-workout routine. It's plant-based, blends well, and doesn't leave a chalky aftertaste.",
        "These silicone food bags are a fantastic alternative to disposable plastic bags. They're leak-proof, easy to clean, and I feel great about reducing my waste.",
        "This smart scale goes beyond just weight measurement. It tracks various health metrics and syncs seamlessly with my fitness app, helping me stay on track with my goals."





    ],
    "target": [0] * 50
}

In [269]:
len(data2['text_'])

50

In [270]:
test_dataset_raw_l_50 = pd.DataFrame(data2)

In [271]:
test_dataset_raw_4_100 = test_dataset_raw_1_100[['text_','target']]

In [272]:
merged_df_120_4 = pd.concat([test_dataset_raw_l_50, test_dataset_raw_4_100], ignore_index=True)

# Shuffle the index
test_120_4= merged_df_120_4.sample(frac=1, random_state=42).reset_index(drop=True)

In [273]:
test_encodings_120_4 = tokenizer(test_120_4['text_'].tolist(), truncation=True, padding=True, max_length=256)

test_dataset_120_4 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_120_4),
    test_120_4['target'].values
))

In [274]:
test_dataset_120_4_batched = test_dataset_120_4.batch(16)
y_pred_raw_120_4 = model.predict(test_dataset_120_4_batched)

# Convert raw predictions to class predictions
y_pred_120_4 = np.argmax(y_pred_raw_120_4.logits, axis=1)

# Get the actual class labels
y_true_120_4 = []
for features, label in test_dataset_120_4:
    y_true_120_4.append(label.numpy())
y_true_120_4 = np.array(y_true_120_4)


# Generate a classification report
report_120_4 = classification_report(y_true_120_4, y_pred_120_4, target_names=["CG", "OR"],digits =4)
print(report_120_4)

10/10 [==============================] - 3s 248ms/step
              precision    recall  f1-score   support

          CG     0.9057    0.9600    0.9320        50
          OR     0.9794    0.9500    0.9645       100

    accuracy                         0.9533       150
   macro avg     0.9425    0.9550    0.9483       150
weighted avg     0.9548    0.9533    0.9537       150

